In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Fetch Data 

In [2]:
df = pd.read_csv("./SalesFINAL12312016.csv")
df.head()

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/1/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,1/2/2016,750,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/3/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,1/8/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,1/9/2016,375,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [3]:
df.shape

(1048575, 14)

This implies we have 1,048,575 (near a million) rows and 14 columns(features) in our dataframe.

To perform **Demand Forecasting**, we will take a random sample from the dataframe, of the size = 80,000

In [4]:
df = df.sample(n = 80000)

## DATA ANALYSIS

In [5]:
print("\nSales Columns:")
print(df.columns.tolist())


Sales Columns:
['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification', 'ExciseTax', 'VendorNo', 'VendorName']


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80000 entries, 822746 to 146008
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   InventoryId     80000 non-null  object 
 1   Store           80000 non-null  int64  
 2   Brand           80000 non-null  int64  
 3   Description     80000 non-null  object 
 4   Size            80000 non-null  object 
 5   SalesQuantity   80000 non-null  int64  
 6   SalesDollars    80000 non-null  float64
 7   SalesPrice      80000 non-null  float64
 8   SalesDate       80000 non-null  object 
 9   Volume          80000 non-null  int64  
 10  Classification  80000 non-null  int64  
 11  ExciseTax       80000 non-null  float64
 12  VendorNo        80000 non-null  int64  
 13  VendorName      80000 non-null  object 
dtypes: float64(3), int64(6), object(5)
memory usage: 9.2+ MB


In [7]:
df.dtypes

InventoryId        object
Store               int64
Brand               int64
Description        object
Size               object
SalesQuantity       int64
SalesDollars      float64
SalesPrice        float64
SalesDate          object
Volume              int64
Classification      int64
ExciseTax         float64
VendorNo            int64
VendorName         object
dtype: object

**Observation**: We observe that, the SalesDate feature is of type "object". We need SalesDate later, to predict the SalesQuantity, so we need to handle it's data type.

In [8]:
df.describe()

,Store,Brand,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,40.178100,12164.435788,2.337912,31.599391,15.431885,944.451600,1.416688,1.311377,6993.241337
std,24.403392,12392.136440,3.496596,72.888603,21.830694,720.201131,0.493013,3.292489,8400.738804
min,1.000000,58.000000,1.000000,0.990000,0.490000,50.000000,1.000000,0.010000,105.000000
25%,15.000000,3670.000000,1.000000,10.990000,8.990000,750.000000,1.000000,0.160000,3252.000000
50%,39.000000,6296.000000,1.000000,17.990000,12.990000,750.000000,1.000000,0.680000,4425.000000
75%,64.000000,17957.000000,2.000000,31.990000,18.990000,1000.000000,2.000000,1.570000,9552.000000
max,79.000000,90089.000000,247.000000,7093.710000,4999.990000,20000.000000,2.000000,194.510000,173357.000000


**Observation**: We observe that there is a huge deviation between our minimum and maximum SalesQuantity. This will also affect our predictions. We will handle these extreme values(*Outliers*), later in this notebook.

In [9]:
df.isnull().sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
dtype: int64

**Observation**: We don't have any null values in our DataFrame. So far good to GO!

In [10]:
df.Description.value_counts()

Jack Daniels No 7 Black         552
Capt Morgan Spiced Rum          524
Smirnoff 80 Proof               519
Absolut 80 Proof                516
Jagermeister Liqueur            512
                               ... 
Ch Pichon Longville 12 Pauil      1
Don Eduardo Anejo Tequila         1
L'Ecole No 41 Chenin Bl OV        1
Ch Croix de Jaugue St Emilio      1
Blackstone Znfdl Cal              1
Name: Description, Length: 4620, dtype: int64

**Observation**: In our dataframe, for some brands, we've enough number of rows to predict their sales, but for some brands we only have one row, which may impact our predictions.

## DATA PREPROCESSING

In [11]:
df['VendorName'] = df['VendorName'].str.strip()
df['Description'] = df['Description'].str.strip()

Convert the SalesDate to Date time Object

In [12]:
df['SalesDate'] = pd.to_datetime(df['SalesDate'])

In [13]:
df.dtypes

InventoryId               object
Store                      int64
Brand                      int64
Description               object
Size                      object
SalesQuantity              int64
SalesDollars             float64
SalesPrice               float64
SalesDate         datetime64[ns]
Volume                     int64
Classification             int64
ExciseTax                float64
VendorNo                   int64
VendorName                object
dtype: object

Extract the year, month and day from the SalesDate

In [14]:
df['year'] = df['SalesDate'].dt.year
df['month'] = df['SalesDate'].dt.month
df['day'] = df['SalesDate'].dt.day

In [15]:
df

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName,year,month,day
822746,73_DONCASTER_5330,73,5330,Kahlua White Russian RTD,1.75L,1,17.99,17.99,2016-01-15,1750,1,1.84,17035,PERNOD RICARD USA,2016,1,15
394928,43_WOLFORD_1892,43,1892,Yukon Jack,50mL,10,9.90,0.99,2016-01-23,50,1,0.52,3960,DIAGEO NORTH AMERICA INC,2016,1,23
1040784,17_OLDHAM_8476,17,8476,1800 Reposado Gold Tequila,375mL,1,12.49,12.49,2016-02-20,375,1,0.39,7245,PROXIMO SPIRITS INC.,2016,2,20
38535,11_CARDEND_22584,11,22584,Cambria Katherines Chard,750mL,3,53.97,17.99,2016-01-04,750,2,0.34,9552,M S WALKER INC,2016,1,4
792390,72_HARDERSFIELD_4234,72,4234,Bacardi Pineapple Fusion,750mL,1,13.99,13.99,2016-01-07,750,1,0.79,480,BACARDI USA INC,2016,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509897,54_GUTHRAM_3913,54,3913,Svedka Vodka,1.75L,1,17.99,17.99,2016-01-07,1750,1,1.84,1392,CONSTELLATION BRANDS INC,2016,1,7
906756,79_BALLYMENA_3388,79,3388,Seagrams Extra Dry Gin,750mL,1,9.99,9.99,2016-01-09,750,1,0.79,17035,PERNOD RICARD USA,2016,1,9
1018125,15_WANBORNE_34175,15,34175,Lindemans Bin 50 Shiraz,1.5L,2,17.98,8.99,2016-02-21,1500,2,0.45,4425,MARTIGNETTI COMPANIES,2016,2,21
278835,34_PITMERDEN_2426,34,2426,The Glenlivet 15Yr FrenchOak,750mL,1,54.99,54.99,2016-01-25,750,1,0.79,17035,PERNOD RICARD USA,2016,1,25


Drop the following Columns:
1) InventoryId: because each row has different ID 
2) SalesDate: because we've already extracted the required info from this feature in year, month, and day columns.
3) Size: because it has some ambiguity in it. 
4) SalesDollars: represents redundant data in terms of SalesPrice * SalesQuantity

In [16]:
df.drop(['InventoryId','SalesDate', 'Size','SalesDollars'], inplace=True, axis=1)

In [17]:
df.head()

,Store,Brand,Description,SalesQuantity,SalesPrice,Volume,Classification,ExciseTax,VendorNo,VendorName,year,month,day
822746,73,5330,Kahlua White Russian RTD,1,17.99,1750,1,1.84,17035,PERNOD RICARD USA,2016,1,15
394928,43,1892,Yukon Jack,10,0.99,50,1,0.52,3960,DIAGEO NORTH AMERICA INC,2016,1,23
1040784,17,8476,1800 Reposado Gold Tequila,1,12.49,375,1,0.39,7245,PROXIMO SPIRITS INC.,2016,2,20
38535,11,22584,Cambria Katherines Chard,3,17.99,750,2,0.34,9552,M S WALKER INC,2016,1,4
792390,72,4234,Bacardi Pineapple Fusion,1,13.99,750,1,0.79,480,BACARDI USA INC,2016,1,7


### Removing the Outliers

In [18]:
z_scores = (df[['SalesQuantity', 'SalesPrice', 'Volume', 'ExciseTax']] - df[['SalesQuantity', 'SalesPrice', 'Volume', 'ExciseTax']].mean()) / df[['SalesQuantity', 'SalesPrice', 'Volume', 'ExciseTax']].std()
threshold = 3
outliers = df[(np.abs(z_scores) > threshold).any(axis=1)]
df_cleaned = df[(np.abs(z_scores) <= threshold).all(axis=1)]

In [19]:
df_cleaned

,Store,Brand,Description,SalesQuantity,SalesPrice,Volume,Classification,ExciseTax,VendorNo,VendorName,year,month,day
822746,73,5330,Kahlua White Russian RTD,1,17.99,1750,1,1.84,17035,PERNOD RICARD USA,2016,1,15
394928,43,1892,Yukon Jack,10,0.99,50,1,0.52,3960,DIAGEO NORTH AMERICA INC,2016,1,23
1040784,17,8476,1800 Reposado Gold Tequila,1,12.49,375,1,0.39,7245,PROXIMO SPIRITS INC.,2016,2,20
38535,11,22584,Cambria Katherines Chard,3,17.99,750,2,0.34,9552,M S WALKER INC,2016,1,4
792390,72,4234,Bacardi Pineapple Fusion,1,13.99,750,1,0.79,480,BACARDI USA INC,2016,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
509897,54,3913,Svedka Vodka,1,17.99,1750,1,1.84,1392,CONSTELLATION BRANDS INC,2016,1,7
906756,79,3388,Seagrams Extra Dry Gin,1,9.99,750,1,0.79,17035,PERNOD RICARD USA,2016,1,9
1018125,15,34175,Lindemans Bin 50 Shiraz,2,8.99,1500,2,0.45,4425,MARTIGNETTI COMPANIES,2016,2,21
278835,34,2426,The Glenlivet 15Yr FrenchOak,1,54.99,750,1,0.79,17035,PERNOD RICARD USA,2016,1,25


In [20]:
df_cleaned.shape

(76750, 13)

**Observation** : Earlier we had 80,000 rows, after removing the outliers, the number of rows has reduced.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80000 entries, 822746 to 146008
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Store           80000 non-null  int64  
 1   Brand           80000 non-null  int64  
 2   Description     80000 non-null  object 
 3   SalesQuantity   80000 non-null  int64  
 4   SalesPrice      80000 non-null  float64
 5   Volume          80000 non-null  int64  
 6   Classification  80000 non-null  int64  
 7   ExciseTax       80000 non-null  float64
 8   VendorNo        80000 non-null  int64  
 9   VendorName      80000 non-null  object 
 10  year            80000 non-null  int64  
 11  month           80000 non-null  int64  
 12  day             80000 non-null  int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 8.5+ MB


### Applying One-Hot Encoding
To enable the model to effectively process textual data (*product descriptions*), one-hot encoding is applied. This process converts description strings into individual boolean columns, where each column represents a specific type of product. 

In [22]:
description_dummies = pd.get_dummies(df_cleaned['Description'])
vendorname_dummies = pd.get_dummies(df_cleaned['VendorName'])
dfx = pd.concat([df_cleaned, description_dummies, vendorname_dummies], axis=1)
dfx.drop(['Description', 'VendorName'], axis=1, inplace=True)
dfx.head()

,Store,Brand,SalesQuantity,SalesPrice,Volume,Classification,ExciseTax,VendorNo,year,month,...,VINEXTRA INC,VINEYARD BRANDS INC,VINILANDIA USA,VRANKEN AMERICA,WALPOLE MTN VIEW WINERY,WEIN BAUER INC,WESTERN SPIRITS BEVERAGE CO,WILLIAM GRANT & SONS INC,WINE GROUP INC,ZORVINO VINEYARDS
822746,73,5330,1,17.99,1750,1,1.84,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0
394928,43,1892,10,0.99,50,1,0.52,3960,2016,1,...,0,0,0,0,0,0,0,0,0,0
1040784,17,8476,1,12.49,375,1,0.39,7245,2016,2,...,0,0,0,0,0,0,0,0,0,0
38535,11,22584,3,17.99,750,2,0.34,9552,2016,1,...,0,0,0,0,0,0,0,0,0,0
792390,72,4234,1,13.99,750,1,0.79,480,2016,1,...,0,0,0,0,0,0,0,0,0,0


The number of columns in our data has increased, reflecting the one-hot encoding

In [23]:
dfx

,Store,Brand,SalesQuantity,SalesPrice,Volume,Classification,ExciseTax,VendorNo,year,month,...,VINEXTRA INC,VINEYARD BRANDS INC,VINILANDIA USA,VRANKEN AMERICA,WALPOLE MTN VIEW WINERY,WEIN BAUER INC,WESTERN SPIRITS BEVERAGE CO,WILLIAM GRANT & SONS INC,WINE GROUP INC,ZORVINO VINEYARDS
822746,73,5330,1,17.99,1750,1,1.84,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0
394928,43,1892,10,0.99,50,1,0.52,3960,2016,1,...,0,0,0,0,0,0,0,0,0,0
1040784,17,8476,1,12.49,375,1,0.39,7245,2016,2,...,0,0,0,0,0,0,0,0,0,0
38535,11,22584,3,17.99,750,2,0.34,9552,2016,1,...,0,0,0,0,0,0,0,0,0,0
792390,72,4234,1,13.99,750,1,0.79,480,2016,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509897,54,3913,1,17.99,1750,1,1.84,1392,2016,1,...,0,0,0,0,0,0,0,0,0,0
906756,79,3388,1,9.99,750,1,0.79,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0
1018125,15,34175,2,8.99,1500,2,0.45,4425,2016,2,...,0,0,0,0,0,0,0,0,0,0
278835,34,2426,1,54.99,750,1,0.79,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0



A value of true in the column indicates that the respective row refers to that product.

### Applying Min-Max Scaling
Min-Max Scaling is done to linearly scale the values of *Sales Dollars* and *Sales Price* between 0 and 1.

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfx.loc[:, "SalesPrice"] = scaler.fit_transform(df_cleaned[["SalesPrice"]].values.reshape(-1, 1))

In [25]:
dfx

,Store,Brand,SalesQuantity,SalesPrice,Volume,Classification,ExciseTax,VendorNo,year,month,...,VINEXTRA INC,VINEYARD BRANDS INC,VINILANDIA USA,VRANKEN AMERICA,WALPOLE MTN VIEW WINERY,WEIN BAUER INC,WESTERN SPIRITS BEVERAGE CO,WILLIAM GRANT & SONS INC,WINE GROUP INC,ZORVINO VINEYARDS
822746,73,5330,1,0.220126,1750,1,1.84,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0
394928,43,1892,10,0.006289,50,1,0.52,3960,2016,1,...,0,0,0,0,0,0,0,0,0,0
1040784,17,8476,1,0.150943,375,1,0.39,7245,2016,2,...,0,0,0,0,0,0,0,0,0,0
38535,11,22584,3,0.220126,750,2,0.34,9552,2016,1,...,0,0,0,0,0,0,0,0,0,0
792390,72,4234,1,0.169811,750,1,0.79,480,2016,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509897,54,3913,1,0.220126,1750,1,1.84,1392,2016,1,...,0,0,0,0,0,0,0,0,0,0
906756,79,3388,1,0.119497,750,1,0.79,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0
1018125,15,34175,2,0.106918,1500,2,0.45,4425,2016,2,...,0,0,0,0,0,0,0,0,0,0
278835,34,2426,1,0.685535,750,1,0.79,17035,2016,1,...,0,0,0,0,0,0,0,0,0,0


## PREDICTING SALES DEMAND

In [26]:
y=dfx['SalesQuantity']
x=dfx.drop(['SalesQuantity'],axis=1)

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [28]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()

Training the model

In [29]:
model.fit(x_train,y_train)

RandomForestClassifier()

Evaluating the model

In [30]:
model.score(x_test,y_test)

0.88878465707734

In [31]:
import warnings
warnings.filterwarnings('ignore')

This function returns the index of the column associated with the provided description after one hot encoding

In [32]:
def get_loc_index(description):
    try:
        return x.columns.get_loc(description)
    except KeyError:
        return -1 

In [33]:
def predict_sales_quantity(VendorName, Description, Store, Brand, Volume, SalesPrice, Classification, ExciseTax, VendorNo,year,month,day):
    X = np.zeros(len(x.columns))
    X[0] = Store
    X[1] = Brand
    X[2] = Volume
    X[3] = SalesPrice
    X[4] = Classification
    X[5] = ExciseTax
    X[6] = VendorNo
    X[7] = year
    X[8] = month
    X[9]= day

    # Set the index corresponding to Description column to 1 if loc_index >= 0
    loc_index = get_loc_index(Description)
    if loc_index >= 0:
        X[loc_index] = 1

    # Make prediction
    predicted_sales_quantity = model.predict([X])[0]

    return predicted_sales_quantity

In [35]:
predict_sales_quantity('ULTRA BEVERAGE COMPANY LLP','Cecchi Sangiovese',1,18013,750,6.99,2,0.56,9165,2017,1,21)

5